## Homophobic lyrics detection - Binary Classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/HOMO-MEX/task_3')

In [ ]:
from config import config

In [ ]:
%%capture
#! pip install --quiet "torchvision" "torch>==1.10" "pytorch-lightning>=1.3" "torchmetrics>=0.3" "typing-extensions<4,>=3.7.4.3" "tf-estimator-nightly==2.8.0.dev2021122109" "folium==0.2.1"
! pip install torchmetrics

In [ ]:
%%capture
!pip install transformers # pre-trained models from https://huggingface.co/

### Import all the libraries and functions we will use

In [ ]:
import re
import os
import time
import datetime
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

from transformers import BertModel
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler

from torchmetrics.classification import MulticlassF1Score, BinaryF1Score

pd.set_option('display.max_colwidth',None)

### Environment set up


Select the device where the model is to be trained.

In [ ]:
run_on = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = torch.device(run_on)
DEVICE

device(type='cuda')

Set a random seed to ensure that the results are reproducible when attempting to run this *notebook* again.

In [ ]:
# set random seed
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed_all(value)

set_seed(config.RANDOM_STATE)

### Dataset

In [ ]:
class CustomDataset:
  def __init__(self, datapath, version, batch_size, modelname, test_size=0.2):
    self.datapath = datapath
    self.batch_size = batch_size
    self.dataloaders = {}
    self.dfs = {}
    self.tokenizer = BertTokenizerFast.from_pretrained(modelname
                                          ,do_lower_case=(True if 'uncased' in modelname else False))
    self.prepareData(version)

  def prepareData_kfold(self):
    df = pd.read_csv(self.datapath, converters={ config.TEXT:str, config.TARGET:str })
    df.drop(columns=["Unnamed: 0"], inplace=True)
    df["label"] = df["label"].map({ "P": 0, "NP": 1 })

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=config.RANDOM_STATE)
    kfolds = skf.split(df[config.TEXT], df[config.TARGET])

    for i, (train_index, test_index) in enumerate(kfolds):
      train = df.iloc[list(train_index)]
      test = df.iloc[list(test_index)]

      train = train.sample( len(train), random_state=config.RANDOM_STATE )
      test = test.sample( len(test), random_state=config.RANDOM_STATE )

      self.dfs[i] = { "train": train, "val": test }
      self.dataloaders[i] = { "train": self.getTensorDataset(train), "val": self.getTensorDataset(test) }

    return self.dataloaders

  def prepareData(self, version):
    dataset_names = [ "train", "dev", "test" ]
    for key in dataset_names:
      df = pd.read_csv(f"{self.datapath}{key}_{version}.csv", converters={ config.TEXT:str })
      if key == "train":
        # class1 = df[ df[config.TARGET] == "P"]
        # class2 = df[ df[config.TARGET] == "NP"].sample(len(class1), random_state=config.RANDOM_STATE)
        # df = pd.concat([ class1, class2 ])
        df = df.sample( len(df), random_state=config.RANDOM_STATE )

      inputs, masks = self.tokenize(df["text"])

      if config.TARGET in list(df.columns):
        df[config.TARGET] = df[config.TARGET].map({ "P": 0, "NP": 1 })
        labels = torch.tensor(df[config.TARGET].to_numpy(), dtype=torch.long)
        data = TensorDataset(inputs, masks, labels)
      else:
        df[config.ID] = df[config.ID].apply(lambda x: int(x.replace("_Track3", "")))
        ids = torch.tensor(df[config.ID].to_numpy(), dtype=torch.long)
        data = TensorDataset(inputs, masks, ids)

      sampler = SequentialSampler(data)
      self.dfs[key] = df
      self.dataloaders[key] = DataLoader(data,
                                         sampler=sampler,
                                         batch_size=self.batch_size,
                                         num_workers=0)
      print("...dataloader for", key, "completed")
    return self.dataloaders

  def getTensorDataset(self, dataset):
    inputs, masks = self.tokenize(dataset[config.TEXT])
    labels = torch.tensor(dataset[config.TARGET].to_numpy(), dtype=torch.float32)
    data = TensorDataset(inputs, masks, labels)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=self.batch_size, num_workers=0)
    return dataloader

  def tokenize(self, dataset):
    input_ids = []
    attention_mask = []

    for doc in dataset:
      encoded_doc = self.tokenizer.encode_plus(doc,
                                          add_special_tokens=True,
                                          max_length=300,
                                          truncation=True,
                                          padding="max_length",
                                          return_token_type_ids=False
                                          )

      input_ids.append(encoded_doc['input_ids'])
      attention_mask.append(encoded_doc['attention_mask'])

    return (torch.tensor(input_ids), torch.tensor(attention_mask))

### Training

This process consists of transforming the words of the messages into features expected by BETO.


In [ ]:
# PARAMETERS
import math

# Model
NUM_LABELS = 2
TARGET_NAMES = ["P", "NP"]

# Training
BATCH_SIZE = 8
GRADIENT_ACCUMULATOR_SIZE = 1
NUM_EPOCHS = 10

GET_ATTENTIONS = False
GET_HIDDEN_STATES = False
DROPOUT = 0.01

# Optimizer
LR = 2e-5
EPS = 1e-10
WEIGHT_DECAY = 0.01
BETAS = (0.9, 0.999)
AMSGRAD = False

# Loss
REDUCTION = "sum"
LABEL_SMOOTHING = 0

# Scheduler
POWER = 1.0

In [ ]:
#fuction to format time
def format_time(elapsed):
  elapsed_rounded = int(round((elapsed)))
  return str(datetime.timedelta(seconds=elapsed_rounded))

#function to clean cuda memory
def clean_cuda_memory(iterable_var):
  for elem in iterable_var:
    elem.to('cpu')
    del elem
  torch.cuda.empty_cache()
  gc.collect()

def get_class_weights(classes_list, labels):
  return compute_class_weight('balanced', classes=classes_list, y=labels)

#variable to plot the confusion matrix
conf = []

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, modelpath, freeze_model=False, num_labels=2, get_att=False, get_hs=False, dropout=0.05):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(modelpath
                                              ,num_labels=num_labels
                                              ,output_attentions=get_att
                                              ,output_hidden_states=get_hs)

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.freeze_model = freeze_model

        if self.freeze_model:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids
                            ,token_type_ids=None
                            ,attention_mask=attention_mask)

        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

In [ ]:
class CustomModel:
  def __init__(self, modelpath, class_weights):
    self.model = TextClassifier(modelpath, num_labels=NUM_LABELS, get_att=GET_ATTENTIONS, get_hs=GET_HIDDEN_STATES, dropout=DROPOUT)

    self.optimizer = torch.optim.Adam(self.model.parameters()
                                      ,lr = LR
                                      ,eps = EPS
                                      ,weight_decay = WEIGHT_DECAY
                                      ,betas = BETAS # 0.9, 0.999
                                      ,amsgrad = AMSGRAD)

    self.criterion = nn.CrossEntropyLoss(weight=class_weights # torch.tensor([5.8, 0.43, 2]).to(device)
                                         ,reduction = REDUCTION #sum
                                         ,label_smoothing = LABEL_SMOOTHING)

  def freeze_all(self):
    #Freeze all layers except classifier
    for name, param in self.model.named_parameters():
      if "encoder.layer" in name:
        param.requires_grad = False

  def unfreeze(self, layers=np.arange(0,13,1)):
    self.freeze_all()
    #Unfreeze specified layers
    for layer_no in layers:
      for name, param in self.model.named_parameters():
        if "encoder.layer" in name and str(layer_no) in name:
          param.requires_grad = True

  def freeze_embs(self):
    for name, param in self.model.named_parameters():
      if "embeddings" in name:
        param.requires_grad = False

  def validation(self, val_dataloader):
    t0 = time.time()

    # We put the model in validation mode
    self.model.eval()

    # We declare variables
    eval_loss = 0
    eval_metric = 0
    all_logits = []
    all_labels = []

    # By minibatches
    for step, batch in enumerate(val_dataloader):
      b_input_ids, b_input_mask, b_labels = tuple(t.to(DEVICE) for t in batch)

      with torch.no_grad():
        # We generate the predictions of the model
        outputs = self.model(b_input_ids,
                            attention_mask=b_input_mask)

        loss = self.criterion(outputs, b_labels)

        # ...we extract them
        logits = torch.argmax(outputs, dim=1).detach().cpu()
        b_labels = b_labels.to('cpu')

        # Saving logits and labels. They will be useful for the confusion matrix.
        #predict_labels = np.argmax(logits, axis=1).flatten()
        all_logits.extend(logits.tolist())
        all_labels.extend(b_labels.tolist())

        eval_loss += loss

    # We calculate the F1 score of this batch
    scores = MulticlassF1Score(num_classes=2, average=None)(torch.tensor(all_logits), torch.tensor(all_labels))
    score = BinaryF1Score()(torch.tensor(all_logits), torch.tensor(all_labels))
    # We show the final accuracy for this epoch
    print(f"\n\tF1Scores: {scores.tolist()}")
    print(f"\n\tEvalLoss: {eval_loss}")
    print(f"\tValidation took: {format_time(time.time() - t0)}")
    return scores, eval_loss


  def training(self, n_epochs, train_dataloader, val_dataloader, gradient_accumulator_size=2):
    max_step_t = len(train_dataloader)
    max_step_v = len(val_dataloader)

    scheduler = torch.optim.lr_scheduler.PolynomialLR(self.optimizer
                                                      ,total_iters=n_epochs
                                                      ,power=POWER)

    total_loss = []
    total_lr = []

    for epoch in range(n_epochs):
      if epoch > 7:
        break
      # for each epoch...
      print(f"\nEpoch {epoch + 1} / {n_epochs} :")
      # We save the start time to see how long it takes.
      t0 = time.time()
      # We reset the loss value for each epoch.
      epoch_loss = []

      # Training mode.
      self.model.train()
      self.model.zero_grad()

      for step, batch in enumerate(train_dataloader):
        batch_loss = 0
        b_input_ids, b_input_mask, b_labels = tuple(t.to(DEVICE) for t in batch)

        # Propagation forward in the layers
        outputs = self.model(b_input_ids,
                        attention_mask=b_input_mask)

        # We calculate the loss of the present minibatch
        loss = self.criterion(outputs, b_labels) #outputs[0]
        batch_loss += loss.item()
        epoch_loss.append( loss.item() )

        # Backpropagation
        loss.backward()

        # So we can implement gradien accumulator technique
        if (step > 0 and step % gradient_accumulator_size == 0) or (step == len(train_dataloader) - 1):
          #(this prevents the gradient from becoming explosive)
          torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

          # We update the weights and bias according to the optimizer
          self.optimizer.step()
          # We clean the gradients for the accumulator batch
          self.model.zero_grad()

        b_input_ids.to("cpu")
        b_input_mask.to("cpu")
        b_labels.to("cpu")
        del b_input_ids
        del b_input_mask
        del b_labels
        torch.cuda.empty_cache()
        gc.collect()

        if step % 20 == 0 or step == max_step_t - 1:
          print(f"Batch {step}/{max_step_t} avg loss: {np.sum(epoch_loss) / (step+1):.5f} {np.max(epoch_loss):.5f} {np.min(epoch_loss):.5f}")

      #Update learning rate each end of epoch
      scheduler.step()
      total_lr.append(scheduler.get_last_lr())
      total_loss.append(np.sum(epoch_loss)/len(train_dataloader))

      # We calculate the average loss in the current epoch of the training set
      print(f"\n\tAverage training loss: {np.sum(epoch_loss)/len(train_dataloader):.5f}")
      print(f"\tTraining epoch took: {format_time(time.time() - t0)}")

      print("\n\tValidation")
      curr_score, curr_eval_loss = self.validation(val_dataloader)

    # Display learning rate and loss charts
    """
    epoch_axis = np.arange(0, n_epochs) + 1
    plt.plot(epoch_axis, total_loss)
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.xticks(epoch_axis)
    plt.show()
    """
    """
    print("\n\n")

    plt.plot(epoch_axis, total_lr)
    plt.xlabel("epoch")
    plt.ylabel("learning rate")
    plt.xticks(epoch_axis)
    plt.show()
    """
    print("\nTraining complete")


  def testing(self, dataloader):
    preds = []
    labs = []

    with torch.no_grad():
      for step, batch in enumerate(dataloader):
        test_inputs, test_masks, b_labels = tuple(t.to(DEVICE) for t in batch)

        outputs = self.model(test_inputs,
                             attention_mask=test_masks)

        logits = torch.argmax(outputs, dim=1).detach().cpu()
        b_labels = b_labels.to('cpu').tolist()

        preds.extend([x.item() for x in logits])
        labs.extend(b_labels)

    preds = [ "P" if pred == 0 else "NP" for pred in preds ]
    labs = [ str(l) + "_Track3" for l in labs ]

    dataframe_res = { "sub_id": labs, "label": preds }
    dataframe_res = pd.DataFrame.from_dict(dataframe_res)

    return dataframe_res

  def metrics_testing(self, dataloader):
    preds = []
    labs = []

    with torch.no_grad():
      for step, batch in enumerate(dataloader):
        test_inputs, test_masks, b_labels = tuple(t.to(DEVICE) for t in batch)

        outputs = self.model(test_inputs,
                             attention_mask=test_masks)

        logits = torch.argmax(outputs, dim=1).detach().cpu()
        b_labels = b_labels.to('cpu').tolist()

        preds.extend([x.item() for x in logits])
        labs.extend(b_labels)

    return preds, labs


In [ ]:
from sklearn.metrics import classification_report

def report_metrics(data):
  preds, targets = data[0], data[1]
  print(classification_report(targets, preds, target_names=TARGET_NAMES, digits=4))

In [ ]:
import gc

def fine_tuning_kfold(layers, desc, modelname, num_epochs = 10, embeddings = True, gradient_accumulator_size = 4):
  for kfold in datasets.dataloaders:
    if kfold > 2:
      break

    train = datasets.dataloaders[kfold]["train"]
    dev = datasets.dataloaders[kfold]["val"]
    train_count = datasets.dfs[kfold]["train"][config.TEXT].count()
    dev_count = datasets.dfs[kfold]["val"][config.TEXT].count()
    print(f"K-Fold: {kfold} -->  train size:{ train_count } val size:{ dev_count }")
    print("------------------------")

    class_weights = get_class_weights([0,1], datasets.dfs[kfold]["train"][config.TARGET])
    print(f"class weights: {class_weights}")
    class_weights = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
    m = CustomModel(modelname, class_weights)
    m.unfreeze(layers)

    if not embeddings:
      m.freeze_embs()

    m.model.to(DEVICE)
    m.training(num_epochs, train, dev)

    # m.model.save_pretrained(f"{config.MODELPATH}{config.TRAINED_BETO}_{str(kfold)}")
    print("Training complete. Validation results:\n")
    report_metrics(m.testing(dev))

    m.model.to("cpu")
    del m
    torch.cuda.empty_cache()
    gc.collect()

#TODO : save best model and prepare test results
def fine_tuning(layers, desc, modelname, num_epochs = 10, embeddings = True, gradient_accumulator_size = 4):
  train = datasets.dataloaders["train"]
  dev = datasets.dataloaders["dev"]
  test = datasets.dataloaders["test"]

  train_count = datasets.dfs["train"][config.TEXT].count()
  dev_count = datasets.dfs["dev"][config.TEXT].count()

  print(f"train_size:{ train_count }   val_size:{ dev_count }")
  print("------------------------")

  class_weights = get_class_weights([0,1], datasets.dfs["train"][config.TARGET])
  class_weights[0] = class_weights[0] * 2
  print(f"class weights: {class_weights}")
  class_weights = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
  m = CustomModel(modelname, class_weights)
  m.unfreeze(layers)

  if not embeddings:
    m.freeze_embs()

  m.model.to(DEVICE)
  m.training(num_epochs, train, dev)
  df_res = m.testing(test)
  df_res.to_csv(config.TRACK3 + "tests/test_beto.csv")
  # m.model.save_pretrained(f"{config.MODELPATH}{config.TRAINED_BETO}_{str(kfold)}")
  print("Training complete. Validation results:\n")
  report_metrics(m.metrics_testing(dev))

  m.model.to("cpu")
  del m
  torch.cuda.empty_cache()
  gc.collect()

In [ ]:
#Obtener datasets
datasets = CustomDataset(config.TRACK3,
                         version = "v6",
                         batch_size = BATCH_SIZE,
                         modelname = config.C_BETO)

...dataloader for train completed
...dataloader for dev completed
...dataloader for test completed


In [ ]:
datasets.dfs["train"][config.TARGET].value_counts()

label
1    904
0     39
Name: count, dtype: int64

In [ ]:
layers = [str(x) for x in range(0, 12)]
fine_tuning(layers, "all_layers_cased", config.C_BETO, num_epochs=NUM_EPOCHS + 10, gradient_accumulator_size=GRADIENT_ACCUMULATOR_SIZE)

train_size:943   val_size:600
------------------------
class weights: [24.17948718  0.5215708 ]


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1 / 20 :
Batch 0/118 avg loss: 17.02647 17.02647 17.02647
Batch 20/118 avg loss: 8.54996 25.15743 3.05301
Batch 40/118 avg loss: 8.35806 44.23321 1.80532
Batch 60/118 avg loss: 9.30191 62.09171 0.97837
Batch 80/118 avg loss: 14.15950 103.15755 0.44690
Batch 100/118 avg loss: 13.70594 103.15755 0.26053
Batch 117/118 avg loss: 13.00067 103.15755 0.08690

	Average training loss: 13.00067
	Training epoch took: 0:01:15

	Validation

	F1Scores: [0.0, 0.9655172228813171]

	EvalLoss: 3991.72998046875
	Validation took: 0:00:10

Epoch 2 / 20 :
Batch 0/118 avg loss: 90.97217 90.97217 90.97217
Batch 20/118 avg loss: 39.49754 195.44598 0.02472
Batch 40/118 avg loss: 40.39090 284.61258 0.00989
Batch 60/118 avg loss: 43.88051 305.06778 0.00837
Batch 80/118 avg loss: 53.91994 305.06778 0.00837
Batch 100/118 avg loss: 48.01802 305.06778 0.00837
Batch 117/118 avg loss: 43.40997 305.06778 0.00683

	Average training loss: 43.40997
	Training epoch took: 0:01:15

	Validation

	F1Scores: [0.0, 0.96551